In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt


2025-12-25 16:26:55.468550: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-25 16:26:55.905921: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-25 16:26:57.303081: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
def load_images(folder_path, label, target_size=(100, 100)):
    images, labels = [], []
    for root, _, files in os.walk(folder_path):
        for f in files:
            if f.lower().endswith(('.jpg', '.png', '.jpeg')):
                try:
                    img = Image.open(os.path.join(root, f)).resize(target_size).convert('L')
                    images.append(np.array(img))
                    labels.append(label)
                except Exception as e:
                    print(f"Error reading {f}: {e}")
    return images, labels


In [3]:

base_path = "/home/bhavy/Documents/archive/Industrial-Equipment/"

defected_images, defected_labels = load_images(os.path.join(base_path, "Defected"), 0)
non_defected_images, non_defected_labels = load_images(os.path.join(base_path, "Non-Defected"), 1)

images = np.array(defected_images + non_defected_images)
labels = np.array(defected_labels + non_defected_labels)

images = images / 255.0
images = images.reshape(-1, 100, 100, 1) 

print("Images shape:", images.shape)
print("Labels shape:", labels.shape)


Images shape: (6890, 100, 100, 1)
Labels shape: (6890,)


In [4]:
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)


In [5]:
model = models.Sequential([
    layers.Input(shape=(100, 100, 1)),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


2025-12-25 16:27:25.431004: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 98, 98, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 49, 49, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 47, 47, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 23, 23, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 33856)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     2,166,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,185,729 (8.34 MB)

 Trainable params: 2,185,729 (8.34 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))


Epoch 1/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 13s 74ms/step - accuracy: 0.7114 - loss: 0.5620 - val_accuracy: 0.7569 - val_loss: 0.4625
Epoch 2/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 13s 78ms/step - accuracy: 0.8091 - loss: 0.3938 - val_accuracy: 0.7990 - val_loss: 0.3872
Epoch 3/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 13s 78ms/step - accuracy: 0.8414 - loss: 0.3414 - val_accuracy: 0.8287 - val_loss: 0.3510
Epoch 4/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 12s 72ms/step - accuracy: 0.8641 - loss: 0.2856 - val_accuracy: 0.8157 - val_loss: 0.3877
Epoch 5/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 12s 71ms/step - accuracy: 0.8915 - loss: 0.2239 - val_accuracy: 0.8316 - val_loss: 0.3597
Epoch 6/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 13s 73ms/step - accuracy: 0.9149 - loss: 0.1892 - val_accuracy: 0.8382 - val_loss: 0.3247
Epoch 7/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 13s 77ms/step - accuracy: 0.9363 - loss: 0.1479 - val_accuracy: 0.8498 - val_loss: 0.3257
Epoch 8/10
173/173 ━━━━━━━━━━━━━━━━━━━━ 13s 73ms/step - accuracy: 0.9499 - loss: 0.1199 - 

In [7]:
loss, accuracy = model.evaluate(X_val, y_val)
print("Validation Accuracy:", accuracy)

44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.8679 - loss: 0.3813
Validation Accuracy: 0.8679245114326477


In [8]:
from sklearn.metrics import classification_report

y_pred = (model.predict(X_val) > 0.5).astype(int)
print(classification_report(y_val, y_pred))


44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step
              precision    recall  f1-score   support

           0       0.87      0.88      0.87       724
           1       0.86      0.86      0.86       654

    accuracy                           0.87      1378
   macro avg       0.87      0.87      0.87      1378
weighted avg       0.87      0.87      0.87      1378

